# Customer Airline Company

This dataset contains customer data from an airline company, along with several features that can describe the value of each customer. These features have the potential to provide valuable insights for the airline company to optimize its business strategies, such as marketing, customer retention, or the development of better products and services. By effectively utilizing this dataset, the company can make more informed and accurate decisions in managing its business.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

## Read data

In [ ]:
df = pd.read_csv('flight.csv')
pd.set_option('display.max_columns', None)
df.sample(5)

## EDA

#### Descriptive Statistic

In [ ]:
# check the data type and the number of rows in each column
df.info()

* The column containing the date does not yet have the appropriate data type and the 'MEMBER_NO' column needs to be converted to a string
* Columns 'GENDER', 'WORK_CITY', 'WORK_PROVINCE', 'WORK_COUNTRY', 'AGE', 'SUM_YR_1', 'SUM_YR_2' are indicated to have empty data that needs to be filled

In [ ]:
# change data type
df['MEMBER_NO'] = df['MEMBER_NO'].astype(str)
df['FFP_DATE'] = pd.to_datetime(df['FFP_DATE'], errors='coerce')
df['FIRST_FLIGHT_DATE'] = pd.to_datetime(df['FIRST_FLIGHT_DATE'], errors='coerce')
df['LOAD_TIME'] = pd.to_datetime(df['LOAD_TIME'], errors='coerce')
df['LAST_FLIGHT_DATE'] = pd.to_datetime(df['LAST_FLIGHT_DATE'], errors='coerce')

df.info()


after the data type is changed, there is an additional column that has an empty value which most likely comes from an invalid date, namely the column 'LAST_FLIGHT_DATE' and the data will be dropped

In [ ]:
# drop empty data based on column 'LAST_FLIGHT_DATE'
df.dropna(subset=['LAST_FLIGHT_DATE'], inplace=True)
df.info()

In [ ]:
# check for duplicate data
df['MEMBER_NO'].duplicated().sum()

no duplicate data

In [ ]:
# Group columns by type
cats = ['MEMBER_NO', 'GENDER', 'WORK_CITY', 'WORK_PROVINCE', 'WORK_COUNTRY']
nums = ['FFP_TIER', 'AGE', 'FLIGHT_COUNT', 'BP_SUM', 'SUM_YR_1', 'SUM_YR_2', 'SEG_KM_SUM', 'LAST_TO_END', 'AVG_INTERVAL', 'MAX_INTERVAL',
       'EXCHANGE_COUNT', 'avg_discount', 'Points_Sum', 'Point_NotFlight']
timestamp = ['FFP_DATE', 'FIRST_FLIGHT_DATE', 'LOAD_TIME', 'LAST_FLIGHT_DATE']

In [ ]:
# categorical column statistics
df[cats].describe()

The demographics of the majority of customers are men who live in China, precisely in the city of Guangzhou

In [ ]:
# numeric column statistics
df[nums].describe()

between the Q3 value and the maximum value there is a significant difference so that there are indications of outliers for all numerical features

### Univariate Analysis

In [ ]:
# the number of rows of each unique value categorical column
for col in cats:
    print(f'''Value count kolom {col}:''')
    print(df[col].value_counts())
    print()

In [ ]:
# boxplot for each numeric column
plt.figure(figsize=(12,45))
for i in range(0, len(nums)):
    plt.subplot(len(nums), 5,i+1)
    sns.boxplot(y=df[nums[i]], color='gray', orient='v')
    plt.tight_layout()

indicated that there are outliers in all numeric columns

In [ ]:
# distribution of each numeric column
plt.figure(figsize=(12,45))
for i in range(0, len(nums)):
    plt.subplot(len(nums),5, i+1)
    sns.distplot(df[nums[i]], color='gray')
    plt.tight_layout()

Almost all columns are not normally distributed, except for the 'AGE' and 'avg_discount' columns

In [ ]:
# fill column 'GENDER', 'WORK_CITY', 'WORK_PROVINCE', 'WORK_COUNTRY' with mode (value that appears most often)
df['GENDER'].fillna(df['GENDER'].mode()[0], inplace=True)
df['WORK_CITY'].fillna(df['WORK_CITY'].mode()[0], inplace=True)
df['WORK_PROVINCE'].fillna(df['WORK_PROVINCE'].mode()[0], inplace=True)
df['WORK_COUNTRY'].fillna(df['WORK_COUNTRY'].mode()[0], inplace=True)

In [ ]:
# fill column 'AGE' with average and 'SUM_YR_1', 'SUM_YR_2' with median
df['AGE'].fillna(df['AGE'].mean(), inplace=True)
df['SUM_YR_1'].fillna(df['SUM_YR_1'].median(), inplace=True)
df['SUM_YR_2'].fillna(df['SUM_YR_2'].median(), inplace=True)

In [ ]:
df.info()

now there is no more empty data

In [ ]:
# remove oulier based on IQR because the majority of the data is not normally distributed
print(f'Number of rows before filtering outliers: {len(df)}')

filtered_entries = np.array([True] * len(df))
for col in nums:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    low_limit = Q1 - (IQR * 1.5)
    high_limit = Q3 + (IQR * 1.5)

    filtered_entries = ((df[col] >= low_limit) & (df[col] <= high_limit)) & filtered_entries

df = df[filtered_entries]

print(f'Number of rows after filtering outliers: {len(df)}')

In [ ]:
# encoding of all categorical columns

cats = ['MEMBER_NO', 'GENDER', 'WORK_CITY', 'WORK_PROVINCE', 'WORK_COUNTRY']
df_enc = df.copy()
for i in cats[1:]:
    df_enc[i] = df_enc[i].astype('category').cat.codes

In [ ]:
df_enc.head()

### Multivariate Analysis

In [ ]:
corr = df_enc.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

plt.figure(figsize=(10, 10))
sns.heatmap(corr, mask=mask, cmap='Blues', annot=True, fmt='.2f')

Based on the correlation above, several redundant features are obtained, namely:

* SUM_YR_1 with FLIGHT_COUNT, BP_SUM
* SUM_YR_2 with FLIGHT_COUNT, BP_SUM
* SEG_KM_SUM with FLIGHT_COUNT, BP_SUM, SUM_YR_1, SUM_YR_2
* LAST_TO_END with SUM_YR_2
* MAX_INTERVAL with AVG_INTERVAL
* POINT_SUM with FLIGHT_COUNT, BP_SUM, SUM_YR_1, SUM_YR_2, SEG_KM_SUM

Based on the conditions above, the FLIGHT_COUNT, BP_SUM, SUM_YR_1, SUM_YR_2 and POINT_SUM columns will be dropped to avoid redundant features

there are also features whose correlation values ​​do not appear, namely 'FFP_TIER' and 'EXCHANGE_COUNT' so these features will be removed

### Feature Engineering

In [ ]:
# drop some features
df_new = df_enc.drop(columns=['FLIGHT_COUNT','BP_SUM','SUM_YR_1','SUM_YR_2','FFP_TIER','EXCHANGE_COUNT','Points_Sum',
                          'FFP_DATE', 'FIRST_FLIGHT_DATE', 'LOAD_TIME', 'LAST_FLIGHT_DATE',
                          'MEMBER_NO']).copy()
df_new

There are still 11 features so that dimensionality reduction will then be carried out with PCA


In [ ]:
df_new.columns

In [ ]:
# feature standardization first
feature = ['GENDER', 'WORK_CITY', 'WORK_PROVINCE', 'WORK_COUNTRY', 'AGE',
       'SEG_KM_SUM', 'LAST_TO_END', 'AVG_INTERVAL', 'MAX_INTERVAL',
       'avg_discount', 'Point_NotFlight']
X = df_new.values

from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)
df_std = pd.DataFrame(data = X_std, columns = feature).head()
df_std.describe()

In [ ]:
df_std

In [ ]:
# PCA
from sklearn.decomposition import PCA
pcs = PCA(n_components=4).fit_transform(X_std)
df_pca = pd.DataFrame(data = pcs, columns = ['pc1','pc2','pc3','pc4'])
df_pca.describe()

### Modeling dan Evaluasi

find the number of clusters with the elbow method

In [ ]:
inertia = []

for i in range(1,11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
    kmeans.fit(df_pca.values)
    inertia.append(kmeans.inertia_)

In [ ]:
# visualization of the results of the elbow method
sns.lineplot(x=range(1,11), y=inertia, linewidth=3)
sns.scatterplot(x=range(1,11), y=inertia, s=60)

In [ ]:
(pd.Series(inertia) - pd.Series(inertia).shift(-1)) / pd.Series(inertia) * 100

Based on the elbow method above, 3 clusters are obtained

#### clustering dengan K-means

In [ ]:
# fit model
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=42)
kmeans.fit(df_pca.values)
df_pca['labels_cluster'] = kmeans.labels_

In [ ]:
df_pca

#### cluster evaluation

In [ ]:
# visualization of clustering results
sns.scatterplot(data=df_pca, x='pc1', y='pc2', hue='labels_cluster')

In [ ]:
# calculate the silhouette score
X_sil = df_pca.drop(columns='labels_cluster').values
labels = df_pca['labels_cluster'].values
silhouette_avg = silhouette_score(X_sil, labels)
silhouette_avg

In [ ]:
# silhouette score visualization
sample_silhouette_values = silhouette_samples(X_sil, labels)

n_clusters = 3
y_lower = 10
fig, ax1 = plt.subplots(1, 1)
fig.set_size_inches(6, 4)
ax1.set_xlim([-0.1, 1])
ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

for i in range(n_clusters):
    ith_cluster_silhouette_values = sample_silhouette_values[labels == i]
    ith_cluster_silhouette_values.sort()

    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i

    color = plt.cm.get_cmap("Spectral")(float(i) / n_clusters)
    ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)
    ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    y_lower = y_upper + 10

ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
ax1.set_xlabel("Silhouette Coefficient Values")
ax1.set_ylabel("Cluster Label")
ax1.set_yticks([])
plt.title(("Silhouette analysis for KMeans clustering on sample data "
               "with n_clusters = %d" % n_clusters),
              fontsize=10, fontweight='bold')
plt.show()

In [ ]:
# restore data to original
for col in cats[1:]:
    codes = df_enc[col].unique()
    labels = df[col].unique()
    mapping = dict(zip(codes, labels))
    df_enc[col] = df_enc[col].replace(mapping)

df_enc['labels_cluster'] = kmeans.labels_
df_enc.sample(5)


#### statistical summary

In [ ]:
# numeric column
df_nums = df_enc[nums]
df_nums['label'] = df_enc['labels_cluster']

df_nums.groupby('label').agg(['mean', 'median', 'std'])

In [ ]:
df_cats = df[cats].drop(columns='MEMBER_NO')
df_cats['label'] = df_enc['labels_cluster']

df_cats.groupby('label').agg(pd.Series.mode)

#### based on the statistical summary above, the characteristics of each cluster are obtained as follows:

* Cluster 0: The highest average itinerary, number of flights, total flight distance. For the last flight time distance to the lowest last flight order. Lives in guangzhou city
* Cluster 1: Average itinerary, number of flights, total moderate flight distance, For the distance of the last flight time to the last moderate flight order. Lives in beijing city
* Cluster 2: The lowest average itinerary, number of flights, total flight distance. For the distance between the last flight and the highest last flight order. Lives in guangzhou city

Recommendation:

* Increase the promotion of tour packages in Beijing: Cluster 1 shows that users in Beijing tend to travel medium-distance flights. Therefore, promotion of tour packages with medium-distance destinations around Beijing can be an effective strategy to attract user interest.

* Offering discounts on short-haul flights: Cluster 2 shows that users in Guangzhou city are more likely to book shorter-haul flights. In this case, businesses can offer special discounts on short-haul flights to attract users in Guangzhou city.

* Improve flight service from Guangzhou: Cluster 0 shows that users in Guangzhou city tend to have more compact itineraries and require more flights over longer distances. In this case, businesses can increase flight services from Guangzhou to destinations that are farther away to meet the needs of users in this city.

* Offer accommodation and transportation packages for users in both cities: Based on the clustering results, users in both cities tend to travel with a high number of flights. Therefore, businesses can offer integrated accommodation and transportation packages to make it easier for users to plan their trips.